In [1]:
import pyodbc
import pandas as pd
import numpy as np
import logging

# Set up logging
logging.basicConfig(filename='error_log.log', level=logging.ERROR)

try:
    server = 'IN3508853W1\SQLEXPRESS'
    database = 'final_project'
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
    cursor = cnxn.cursor()
except pyodbc.Error as e:
    logging.error(f"Error connecting to the database: {e}")

# Define a function for reading CSV files with exception handling
def read_csv_with_exception(file_path):
    try:
        df = pd.read_csv(file_path, quotechar="\"")
        df.columns = df.columns.astype(str)
        df.replace({np.inf: np.nan, -np.inf: np.nan}, inplace=True)
        df = df.fillna(0)
        return df
    except FileNotFoundError as e:
        logging.error(f"File not found: {file_path}")
        return pd.DataFrame()  # Return an empty DataFrame in case of file not found
    except pd.errors.EmptyDataError as e:
        logging.error(f"Empty file: {file_path}")
        return pd.DataFrame()  # Return an empty DataFrame in case of an empty file
    except pd.errors.ParserError as e:
        logging.error(f"Error parsing CSV file: {file_path}, {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of a parsing error
    except Exception as e:
        logging.error(f"Unexpected error reading CSV file: {file_path}, {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of unexpected error

# Read CSV files with exception handling
customers_df = read_csv_with_exception(r"C:\finalProjectFiles\Customers.csv")
categories_df = read_csv_with_exception(r"C:\finalProjectFiles\Categories.csv")
employees_df = read_csv_with_exception(r"C:\finalProjectFiles\Employees.csv")
orderDetails_df = read_csv_with_exception(r"C:\finalProjectFiles\orderDetails.csv")
orders_df = read_csv_with_exception(r"C:\finalProjectFiles\Orders.csv")
products_df = read_csv_with_exception(r"C:\finalProjectFiles\Products.csv")
suppliers_df = read_csv_with_exception(r"C:\finalProjectFiles\Suppliers.csv")


In [2]:
suppliers_df.head()

SupplierID                         CompanyName                 ContactName  \
0           1                      Exotic Liquids            Charlotte Cooper   
1           2          New Orleans Cajun Delights               Shelley Burke   
2           3           Grandma Kelly's Homestead               Regina Murphy   
3           4                       Tokyo Traders                Yoshi Nagase   
4           5  Cooperativa de Quesos 'Las Cabras'  Antonio del Valle Saavedra   

           ContactTitle                    Address         City    Region  \
0    Purchasing Manager             49 Gilbert St.       London         0   
1   Order Administrator             P.O. Box 78934  New Orleans        LA   
2  Sales Representative             707 Oxford Rd.    Ann Arbor        MI   
3     Marketing Manager  9-8 Sekimai Musashino-shi        Tokyo         0   
4  Export Administrator          Calle del Rosal 4       Oviedo  Asturias   

  PostalCode Country           Phone  
0    EC1 4SD      UK  (171) 555-2222  
1      70117     USA  (100) 555-4822  
2      48104     USA  (313) 555-5735  
3        100   Japan  (03) 3555-5011  
4      33007   Spain  (98) 598 76 54

In [4]:
load_customers(cnxn, cursor, customers_df)
load_categories(cnxn, cursor, categories_df)
load_employees(cnxn, cursor, employees_df)
load_orderDetails(cnxn, cursor, orderDetails_df)
load_orders(cnxn, cursor, orders_df)
load_products(cnxn, cursor, products_df)
load_suppliers(cnxn, cursor, suppliers_df)

In [3]:
def load_customers(cnxn, cursor, df):
    try:
        cursor.execute("Truncate table north.Customers")
        for index, row in df.iterrows():
            cursor.execute("INSERT INTO final_project.north.Customers(CustomerID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax) values(?,?,?,?,?,?,?,?,?,?,?)", row.CustomerID,row.CompanyName,row.ContactName,row.ContactTitle,row.Address,row.City,row.Region,row.PostalCode,row.Country,row.Phone,row.Fax)
            cnxn.commit()
    except pyodbc.Error as e:
        logging.error(f"Error loading customers data: {e}")

def load_categories(cnxn, cursor, df):
    try:
        cursor.execute("Truncate table north.Categories")
        for index, row in df.iterrows():
            cursor.execute("INSERT INTO north.Categories(CategoryID,CategoryName,Description) values(?,?,?)", row.CategoryID,row.CategoryName,row.Description)
            cnxn.commit()
    except pyodbc.Error as e:
        logging.error(f"Error loading Categories data: {e}")

def load_employees(cnxn, cursor, df):
    try:
        cursor.execute("Truncate table north.Employees")
        for index, row in df.iterrows():
            cursor.execute("INSERT INTO north.Employees(EmployeeID,LastName,FirstName,BirthDate,HireDate,Region,Country) values(?,?,?,?,?,?,?)", row.EmployeeID,row.LastName,row.FirstName,row.BirthDate,row.HireDate,row.Region,row.Country)
            cnxn.commit()
    except pyodbc.Error as e:
        logging.error(f"Error loading Employees data: {e}")
        
def load_orderDetails(cnxn, cursor, df):
    try:
        cursor.execute("Truncate table north.orderDetails")
        for index, row in df.iterrows():
            cursor.execute("INSERT INTO north.orderDetails(orderID,ProductID,unitPrice, Quantity, Discount) values(?,?,?,?,?)", row.OrderID, row.ProductID, row.UnitPrice,row.Quantity,row.Discount)
            cnxn.commit()
    except pyodbc.Error as e:
        logging.error(f"Error loading orderDetails data: {e}")

def load_orders(cnxn, cursor, df):
    try:
        cursor.execute("Truncate table north.Orders")
        for index, row in df.iterrows():
            cursor.execute("INSERT INTO north.Orders(OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry ) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.OrderID,row.CustomerID,row.EmployeeID,row.OrderDate,row.RequiredDate,row.ShippedDate,row.ShipVia, row.Freight, row.ShipName, row.ShipAddress, row.ShipCity, row.ShipRegion, row.ShipPostalCode, row.ShipCountry)
            cnxn.commit()
    except pyodbc.Error as e:
        logging.error(f"Error loading orders data: {e}")

def load_products(cnxn, cursor, df):
    try:
        cursor.execute("Truncate table north.Products")
        for index, row in df.iterrows():
            cursor.execute("INSERT INTO north.Products(ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued) values(?,?,?,?,?,?,?,?,?,?)", row.ProductID,row.ProductName,row.SupplierID,row.CategoryID,row.QuantityPerUnit,row.UnitPrice,row.UnitsInStock, row.UnitsOnOrder, row.ReorderLevel, row.Discontinued)
            cnxn.commit()
    except pyodbc.Error as e:
        logging.error(f"Error loading Products data: {e}")

def load_suppliers(cnxn, cursor, df):
    try:
        cursor.execute("Truncate table north.Suppliers")
        for index, row in df.iterrows():
            cursor.execute("INSERT INTO north.Suppliers(SupplierID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone) values(?,?,?,?,?,?,?,?,?,?)", row.SupplierID,row.CompanyName,row.ContactName,row.ContactTitle,row.Address,row.City,row.Region,row.PostalCode,row.Country,row.Phone)
            cnxn.commit()
    except pyodbc.Error as e:
        logging.error(f"Error loading suppliers data: {e}")
